### NumPy og Pandas er brukt til å beregne gjennomsnitt, median og standardavvik av gjennomsnittsmålingene av havnivåstigningen. 


In [6]:
import numpy as np
import pandas as pd
import requests

# Hent data fra NASA Sea Level API
response = requests.get(
    "https://sealevel-nexus.jpl.nasa.gov/timeSeriesSpark",
    params={
        "ds": "NASA_SSH_REF_SIMPLE_GRID_V1_Monthly",
        "b": "-180.0,-90.0,180.0,90.0",
        "startTime": "1992-01-01T00:00:00Z",
        "endTime": "2025-01-01T00:00:00Z",
        "output": "JSON"
    }
)

# Sjekk at responsen er god
if response.ok and "data" in response.json():
    raw_data = [item for sublist in response.json()["data"] for item in sublist]
    df = pd.DataFrame(raw_data)
else:
    raise ValueError("Feil ved henting av data.")

# Setter en grense for gyldige verdier og filtrer bort ugyldige havnivåverdier 
valid_mean = df['mean'].dropna()
valid_mean = valid_mean[valid_mean.between(-0.1, 0.1)]
invalid_count = len(df) - len(valid_mean)

# Beregn statistiske mål med NumPy
gjennomsnitt = np.mean(valid_mean)
median = np.median(valid_mean)
std_avvik = np.std(valid_mean)

# Print resultatene
print(f"Antall usaklige verdier: {invalid_count}")
print(f"\nGjennomsnittlig havnivåstigning per mnd i meter: {gjennomsnitt}")
print(f"Medianen av havnivåstigningen i meter: {median}")
print(f"Standardavvik av havnivåstigningen i meter: {std_avvik}")


Antall usaklige verdier: 0

Gjennomsnittlig havnivåstigning per mnd i meter: 0.029373403279491547
Medianen av havnivåstigningen i meter: 0.024730028380728995
Standardavvik av havnivåstigningen i meter: 0.02889329956410606


### Undersøker korrelasjon mellom gjennomsnittlig havnivåstigning og tid ved bruk av Pearson-korrelasjon. 
En korrelasjon nær 1 indikerer en sterk positiv sammenheng (havnivået øker over tid). En korrelasjon nær -1 indikerer en sterk negativ sammenheng (havnivået synker). En korrelasjon nær 0 betyr at det ikke er noen tydelig sammenheng.

In [7]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime
from scipy.stats import pearsonr

# Hent data fra API
response = requests.get(
    "https://sealevel-nexus.jpl.nasa.gov/timeSeriesSpark",
    params={
        "ds": "NASA_SSH_REF_SIMPLE_GRID_V1_Monthly",
        "b": "-180.0,-90.0,180.0,90.0",
        "startTime": "1992-01-01T00:00:00Z",
        "endTime": "2025-01-01T00:00:00Z",
        "output": "JSON"
    }
)

if response.ok and "data" in response.json():
    raw_data = [item for sublist in response.json()["data"] for item in sublist]
    df = pd.DataFrame(raw_data)
else:
    raise ValueError("Feil ved henting av data.")

# Forbered og filtrer data
df["iso_time"] = pd.to_datetime(df["iso_time"])
df["år"] = df["iso_time"].dt.year
df["mean_mm"] = df["mean"] * 1000

# Fjern åpenbare feil og interpoler
df = df[df["mean"].between(-1, 1)]
df["mean_mm"] = df["mean_mm"].interpolate()

# Beregn korrelasjon mellom år og havnivå
korrelasjon, _ = pearsonr(df["år"], df["mean_mm"])

# Vis resultat med tolkning
print(f"\nSammenheng mellom år og havnivåstigning (1992–2025): {korrelasjon:.3f}")
if korrelasjon > 0.7:
    print("📈 Sterk positiv sammenheng – havnivået øker over tid.")
elif korrelasjon < -0.7:
    print("📉 Sterk negativ sammenheng – havnivået synker over tid.")
else:
    print("➖ Svak eller ingen klar sammenheng.")



Sammenheng mellom år og havnivåstigning (1992–2025): 0.972
📈 Sterk positiv sammenheng – havnivået øker over tid.
